In [1]:
# Core
import numpy as np
import pandas as pd

# Tensorflow / ML
import tensorflow as tf

# Utility
import os

# Plotting
import matplotlib.pyplot as plt
import matplotlib as mpl

# Utility
import time

In [2]:
# Set visibible GPU
gpu_num: int = 1
os.environ['CUDA_VISIBLE_DEVICES']=f'{gpu_num}'

In [3]:
# Configure TF GPU growth
import kepler_sieve
from tf_utils import gpu_grow_memory, get_gpu_device
gpu_grow_memory(verbose=True)

Found 1 GPUs.  Setting memory growth = True.


In [4]:
# MSE Imports
from asteroid_search import load_fitted_elts, load_ztf_hits, append_fitted_elt, append_ztf_hit
from asteroid_element import load_ast_elt
from candidate_element import asteroid_elts, perturb_elts, random_elts, elts_add_mixture_params, elts_add_H
from random_elements import load_best_random_elts, make_ztf_ast
from ztf_ast import load_ztf_nearest_ast, calc_hit_freq
from ztf_element import load_ztf_batch, make_ztf_batch, ztf_score_by_elt, ztf_elt_summary
from asteroid_model import AsteroidPosition, AsteroidDirection, make_model_ast_pos
from asteroid_search_layers import CandidateElements, MixtureParameters, TrajectoryScore
from asteroid_search_model import AsteroidSearchModel
from asteroid_search_report import traj_diff
from nearest_asteroid import nearest_ast_elt_cart, nearest_ast_elt_cov, elt_q_norm
from element_eda import score_by_elt
from asteroid_dataframe import calc_ast_data, spline_ast_vec_df
from astro_utils import deg2dist, dist2deg, dist2sec

In [5]:
# Aliases
keras = tf.keras

# Constants
dtype = tf.float32
dtype_np = np.float32
space_dims = 3

In [6]:
# Set plot style variables
mpl.rcParams['figure.figsize'] = [16.0, 10.0]
mpl.rcParams['font.size'] = 16

## Review Fitted Elements

In [7]:
known_ast = True

In [59]:
fitted_elts = load_fitted_elts(known_ast=known_ast, min_hits=1)

In [60]:
fitted_elts

,element_id,a,e,inc,Omega,omega,f,epoch,num_hits,R_sec,thresh_sec,log_like,hits,num_rows_close,timestamp
element_id,,,,,,,,,,,,,,,


In [17]:
fitted_elts.loc[25]

element_id                                25
a                                     2.7413
e                                  0.0242049
inc                                0.0982722
Omega                                2.28386
omega                                3.93614
f                                    1.22721
epoch                                  58600
num_hits                             10.0251
R_sec                                1792.82
thresh_sec                              7200
log_like                            0.844332
hits                                       0
num_rows_close                          9014
timestamp         2020-04-23 16:18:58.285763
Name: 25, dtype: object

In [ ]:
# Is this element_id successfully fit? Deem yes if hits > 9.5
is_good_fit = (fitted_elts.hits > 9.5)

# element_ids with good fits
element_id_good = np.unique(fitted_elts.element_id[is_good_fit])
element_id_good

In [ ]:
element_id_good = np.arange(64, dtype=np.int32)

## Review ZTF Hits

In [55]:
ztf_hits = load_ztf_hits(known_ast=known_ast, display=False, min_hits=10)

In [56]:
ztf_hits

,,element_id,ztf_id,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,mag_app,ux,...,vy,vz,elt_ux,elt_uy,elt_uz,elt_r,s_sec,mag_pred,mag_diff,timestamp
element_id,ztf_id,,,,,,,,,,,,,,,,,,,,,
25,5400193,25,5400193,b'ZTF20aaojbhi',1142271234315015006,7890,58896.271238,123.601817,19.796265,18.7572,-0.520713,...,-0.00646,0.001003,-0.520739,0.853715,-0.001027,1.866982,8.487349,19.61643,0.85923,2020-04-23 16:18:58.285763


In [ ]:
ztf_hits.loc[(element_id_good),:]

In [ ]:
ztf_hits.loc[(25, 5400193)]

## Load ZTF Data and Batch of Orbital Elements

In [18]:
# Load orbital elements for known asteroids
ast_elt = load_ast_elt()

# Number of asteroids
N_ast = ast_elt.shape[0]

In [19]:
# Parameters to build random orbital elements and ztf_ast
random_seed = 0
known_ast = True
batch_size_init = 1024
batch_size = 64
thresh_deg = 2.0

In [20]:
# Load ztf nearest asteroid data
ztf_ast = load_ztf_nearest_ast()

In [21]:
# Review ztf_ast
ztf_ast

,ztf_id,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,uz,mag_app,asteroid_prob,nearest_ast_num,nearest_ast_dist,ast_ra,ast_dec,ast_ux,ast_uy,ast_uz
0,0,b'ZTF18acebhfp',676397301515010013,14490,58430.397303,41.357345,58.879488,0.387942,0.653853,0.649598,18.946699,0.865682,1208789,0.005029,41.396388,58.592038,0.390926,0.655662,0.645976
1,1,b'ZTF18abodmwk',596403415715010014,5831,58350.403414,30.969721,65.305308,0.358224,0.558644,0.748059,19.010401,0.855504,1227812,0.024428,33.729101,64.536183,0.357570,0.578168,0.733393
2,2,b'ZTF18abodmwk',626428345715010011,10614,58380.428345,30.969705,65.305294,0.358224,0.558644,0.748059,18.935900,0.855504,1169677,0.015510,29.207596,64.817653,0.371401,0.550472,0.747691
3,3,b'ZTF18abodmwk',630507595715015045,11250,58384.507593,30.969940,65.305305,0.358223,0.558645,0.748059,19.260401,0.855504,1251951,0.012386,30.227911,65.945543,0.352183,0.551505,0.756181
4,4,b'ZTF18abodmwk',618384965715010022,9040,58372.384965,30.969643,65.305179,0.358226,0.558644,0.748058,19.220200,0.855504,1246591,0.025343,34.169666,64.771024,0.352659,0.579473,0.734740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5697957,5697957,b'ZTF20aareruw',1151532523515015015,97109,58905.532523,253.007910,55.485537,-0.165587,-0.169403,0.971537,19.192400,0.608023,1102168,0.036944,253.707834,53.408139,-0.167230,-0.205583,0.964246
5697958,5697958,b'ZTF20aarerwx',1151533002615015009,97110,58905.533009,232.886408,53.509617,-0.358833,-0.115301,0.926253,19.687099,0.559474,1028157,0.084402,224.967815,54.919912,-0.406617,-0.047125,0.912382
5697959,5697959,b'ZTF20aarerww',1151533002115010003,97110,58905.533009,236.167899,54.618457,-0.322375,-0.116973,0.939357,19.957001,0.392662,539940,0.052254,240.693936,56.155104,-0.272611,-0.115218,0.955200
5697960,5697960,b'ZTF20aarervr',1151526063515015015,97098,58905.526065,286.235286,33.876902,0.232120,-0.509626,0.828494,19.049299,0.517241,1246304,0.014054,285.998189,34.657915,0.226704,-0.499251,0.836274


In [22]:
# Best random elements
elts = load_best_random_elts(random_seed=random_seed, known_ast=known_ast, 
                             batch_size_init=batch_size_init, batch_size=batch_size, thresh_deg=thresh_deg)

Loaded random elements in ../data/ztf_elt/random_elts_hit_seed_000_size_64_of_1024_thresh_7200.h5.


In [23]:
# Review best random elements
elts

,element_id,a,e,f,inc,Omega,omega,epoch,num_obs,score,t_score
0,0,3.064054,0.121235,-1.291546,0.068009,0.662207,0.935398,58600.0,12711,998.002460,8.852007
1,1,2.771068,0.151421,-0.531263,0.068958,1.364763,6.111150,58600.0,11901,886.807208,8.129007
2,2,3.172947,0.095819,1.903106,0.035792,3.984825,0.687484,58600.0,12572,781.406566,6.969071
3,3,2.697356,0.098236,0.132486,0.025427,1.610715,5.259525,58600.0,9835,767.421579,7.738322
4,4,3.113809,0.257043,2.596678,0.090822,0.106255,4.320585,58600.0,9129,697.797101,7.303273
...,...,...,...,...,...,...,...,...,...,...,...
59,59,1.890665,0.083441,3.132056,0.035993,0.656159,3.255417,58600.0,9064,280.408751,2.945314
60,60,3.168699,0.088259,0.853523,0.052655,5.188591,0.902693,58600.0,10197,280.102124,2.773832
61,61,2.255124,0.055124,-1.131100,0.065725,4.743550,1.882832,58600.0,6700,279.483345,3.414432
62,62,2.325415,0.176758,-0.619556,0.046713,3.318604,2.235867,58600.0,4532,277.344008,4.119778


## Batches of ZTF Data Near Initial Candidate Elements

In [24]:
# Arguments to make_ztf_batch
near_ast = False
regenerate = False

In [25]:
# Load perturbed element batch
ztf_elt = load_ztf_batch(elts=elts, thresh_deg=thresh_deg, near_ast=near_ast, regenerate=regenerate)

In [26]:
# Review ZTF elements
ztf_elt

,ztf_id,element_id,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,mag_app,ux,...,vz,elt_ux,elt_uy,elt_uz,elt_r,s,s_sec,z,v,is_hit
0,9522,0,b'ZTF18acdjskq',673169444115010000,13711,58427.169444,331.962626,-15.249419,18.609800,0.851563,...,0.000455,0.839983,-0.539036,-0.062191,2.721285,0.021724,4481.021381,0.999764,0.387361,False
1,9535,0,b'ZTF18acdjskq',676148074115010000,14408,58430.148079,331.962675,-15.249472,18.315001,0.851564,...,0.000460,0.842927,-0.534577,-0.060834,2.759572,0.016346,3371.637964,0.999866,0.219306,False
2,52694,0,b'ZTF18abvwxdg',624294433715010000,10156,58378.294433,333.496348,-15.262887,19.549700,0.863341,...,0.000372,0.854746,-0.512130,-0.084457,2.243830,0.020721,4274.139420,0.999785,0.352420,False
3,52695,0,b'ZTF18abvwxdg',621292753715010000,9554,58375.292755,333.496393,-15.262829,19.341499,0.863342,...,0.000367,0.859067,-0.504673,-0.085495,2.229599,0.016570,3417.948771,0.999863,0.225372,False
4,79182,0,b'ZTF18acdjskk',676148074115015019,14408,58430.148079,331.595776,-15.732588,19.734100,0.846661,...,0.000460,0.842927,-0.534577,-0.060834,2.759572,0.009576,1975.281089,0.999954,0.075272,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560175,5682027,63,b'ZTF20aarbsoy',1151130435015015008,96934,58905.130440,46.250023,14.103319,19.349899,0.670669,...,0.000431,0.660978,0.749074,-0.044681,2.592171,0.017045,3515.894844,0.999855,0.238474,False
560176,5682059,63,b'ZTF20aarbsvt',1151130433215015003,96934,58905.130440,46.974922,12.957800,18.811100,0.664943,...,0.000431,0.660978,0.749074,-0.044681,2.592171,0.033797,6971.539164,0.999429,0.937551,False
560177,5682122,63,b'ZTF20aarbsrh',1151130435215015007,96934,58905.130440,44.917822,15.212759,18.506300,0.683306,...,0.000431,0.660978,0.749074,-0.044681,2.592171,0.033002,6807.415135,0.999455,0.893931,False
560178,5682133,63,b'ZTF20aarbsrx',1151130434815015006,96934,58905.130440,47.131113,15.156325,19.110201,0.656659,...,0.000431,0.660978,0.749074,-0.044681,2.592171,0.006666,1374.963549,0.999978,0.036472,False


In [27]:
# Mixture parameters
num_hits: int = 10
R_deg: float = 0.5

In [28]:
# Add mixture parameters to candidate elements
elts_add_mixture_params(elts=elts, num_hits=num_hits, R_deg=R_deg, thresh_deg=thresh_deg)

In [29]:
# Add brightness parameter H
elts_add_H(elts=elts)

In [30]:
# Review perturbed elements; includes nearest asteroid number and distance
elts

,element_id,a,e,f,inc,Omega,omega,epoch,num_obs,score,t_score,num_hits,R,thresh_s,H,sigma_mag
0,0,3.064054,0.121235,-1.291546,0.068009,0.662207,0.935398,58600.0,12711,998.002460,8.852007,10,0.008727,0.034905,16.5,4.0
1,1,2.771068,0.151421,-0.531263,0.068958,1.364763,6.111150,58600.0,11901,886.807208,8.129007,10,0.008727,0.034905,16.5,4.0
2,2,3.172947,0.095819,1.903106,0.035792,3.984825,0.687484,58600.0,12572,781.406566,6.969071,10,0.008727,0.034905,16.5,4.0
3,3,2.697356,0.098236,0.132486,0.025427,1.610715,5.259525,58600.0,9835,767.421579,7.738322,10,0.008727,0.034905,16.5,4.0
4,4,3.113809,0.257043,2.596678,0.090822,0.106255,4.320585,58600.0,9129,697.797101,7.303273,10,0.008727,0.034905,16.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,59,1.890665,0.083441,3.132056,0.035993,0.656159,3.255417,58600.0,9064,280.408751,2.945314,10,0.008727,0.034905,16.5,4.0
60,60,3.168699,0.088259,0.853523,0.052655,5.188591,0.902693,58600.0,10197,280.102124,2.773832,10,0.008727,0.034905,16.5,4.0
61,61,2.255124,0.055124,-1.131100,0.065725,4.743550,1.882832,58600.0,6700,279.483345,3.414432,10,0.008727,0.034905,16.5,4.0
62,62,2.325415,0.176758,-0.619556,0.046713,3.318604,2.235867,58600.0,4532,277.344008,4.119778,10,0.008727,0.034905,16.5,4.0


## Build Asteroid Search Model

In [31]:
# Observatory for ZTF data is Palomar Mountain
site_name = 'palomar'

In [32]:
# Training parameters
learning_rate = 2.0**-12
clipnorm = 1.0
save_at_end: bool = True

In [33]:
# Build asteroid search model
model = AsteroidSearchModel(
                elts=elts, ztf_elt=ztf_elt, 
                site_name=site_name, thresh_deg=thresh_deg, 
                learning_rate=learning_rate, clipnorm=clipnorm,
                name='model',
                file_name=f'candidate_elt_rand_known_{random_seed:04d}.h5',
)

In [34]:
# Load trained model
model.load()

Loaded candidate elements and training history from ../data/candidate_elt/candidate_elt_rand_known_0000.h5.


In [35]:
# Report before training starts
model.report()


Good elements (hits >= 10):   1.00

         \  log_like :  hits  :    R_sec : thresh_sec
Mean Good:   391.70  :  16.00 :    23.52 :   810.18
Mean Bad :    29.25  :   0.11 :   246.65 :  2367.67
Mean     :    34.91  :   0.36 :   243.16 :  2343.34
Median   :    26.56  :   0.00 :   224.45 :  2400.00
GeoMean  :    26.46  :   0.12 :   206.50 :  2325.14
Min      :     6.39  :   0.00 :    23.52 :   810.18
Max      :   391.70  :  16.00 :   691.46 :  2400.00
Trained for 13056 batches over 204 epochs and 60 episodes (elapsed time 585 seconds).


In [36]:
self = model

In [37]:
# Generate fitted elements
elts_fit = self.candidates_df()

In [38]:
cols = ['element_id', 'a', 'e', 'inc', 'Omega', 'omega', 'f', 'epoch',
        'num_hits', 'R', 'R_deg', 'R_sec', 'R_max', 'R_deg_max', 'thresh_s',
        'thresh_deg', 'thresh_sec', 'log_like', 'hits', 'num_rows_close']

In [39]:
elts_fit[cols]

,element_id,a,e,inc,Omega,omega,f,epoch,num_hits,R,R_deg,R_sec,R_max,R_deg_max,thresh_s,thresh_deg,thresh_sec,log_like,hits,num_rows_close
0,0,2.974352,0.097982,0.070416,0.668963,0.934137,-1.314264,58600.0,51.766975,0.000541,0.030983,111.538750,0.005818,0.333333,0.011555,0.662051,2383.382080,47.230007,0.000000,1954.0
1,1,2.747797,0.157316,0.067191,1.346696,6.104029,-0.542777,58600.0,102.445023,0.001292,0.074044,266.558777,0.005818,0.333333,0.011635,0.666667,2399.999756,43.149830,0.000000,1612.0
2,2,2.965837,0.088757,0.047576,3.960974,0.680927,1.889135,58600.0,101.536133,0.001324,0.075832,272.994019,0.005818,0.333333,0.011635,0.666667,2399.999756,34.839916,0.000000,1876.0
3,3,2.761969,0.122120,0.013324,1.628625,5.300218,0.061579,58600.0,82.433159,0.001096,0.062806,226.102524,0.005818,0.333333,0.011635,0.666667,2399.999756,33.437290,0.000000,1501.0
4,4,3.106055,0.246575,0.088836,0.075969,4.334506,2.606801,58600.0,25.510269,0.000719,0.041215,148.373596,0.005818,0.333333,0.011635,0.666667,2399.999756,9.868146,0.000000,1212.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,59,1.879619,0.092083,0.040285,0.685025,3.233119,3.123334,58600.0,42.333218,0.000687,0.039370,141.733093,0.005818,0.333333,0.011635,0.666667,2399.999756,27.697392,0.000000,1318.0
60,60,3.127353,0.087351,0.048705,5.206058,0.873488,0.843077,58600.0,48.521309,0.000593,0.033974,122.306656,0.005818,0.333333,0.011508,0.659343,2373.635010,42.916149,1.717528,1548.0
61,61,2.471154,0.081841,0.069674,4.722207,1.925786,-1.084286,58600.0,63.646912,0.001295,0.074181,267.050232,0.005818,0.333333,0.011635,0.666667,2399.999756,27.185078,0.000000,945.0
62,62,2.313787,0.180719,0.048949,3.254659,2.250012,-0.585662,58600.0,26.032536,0.000519,0.029750,107.100853,0.005818,0.333333,0.007568,0.433596,1560.945923,26.104860,0.000000,269.0


In [40]:
# Save ztf_hits to ztf_hits_known_ast or ztf_unknown_ast

In [41]:
# Generate ZTF hits
ztf_hit = self.calc_ztf_hits()

In [42]:
ztf_hit

,element_id,ztf_id,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,mag_app,ux,...,vx,vy,vz,elt_ux,elt_uy,elt_uz,elt_r,s_sec,mag_pred,mag_diff
0,14,5527114,b'ZTF18aaaiuxx',1146184715115015011,8055,58900.184711,120.449398,21.757523,16.312500,-0.470675,...,-0.005468,-0.006081,0.001213,-0.470697,0.882030,0.021637,2.732104,8.188431,21.034958,4.722458
1,25,1613580,b'ZTF19abyjhox',982490782515015012,2041,58736.490787,115.332837,18.397534,19.536400,-0.406007,...,-0.010250,-0.000864,0.000855,-0.406019,0.912410,-0.051539,3.251177,9.840349,20.846043,1.309643
2,35,2952715,b'ZTF19acnkyqo',1036340636215015009,3797,58790.340637,87.401112,28.640127,18.872900,0.039796,...,-0.008900,0.002506,-0.001381,0.039794,0.995053,0.091022,2.151468,3.320540,20.058588,1.185688
3,35,2953097,b'ZTF19acnkyqo',1036340175115015010,3796,58790.340174,87.401086,28.640113,18.969900,0.039796,...,-0.008900,0.002506,-0.001381,0.039793,0.995053,0.091022,2.151473,3.416393,20.058586,1.088686
4,35,3061820,b'ZTF19aclenna',1033419625115015008,3621,58787.419630,87.632123,28.674719,19.052000,0.036248,...,-0.008871,0.002603,-0.001374,0.036274,0.995139,0.091557,2.182890,5.813280,20.090626,1.038626
5,35,3068346,b'ZTF19aclhbhx',1033505346215015014,3635,58787.505347,87.625723,28.673803,18.355101,0.036347,...,-0.008872,0.002600,-0.001374,0.036373,0.995137,0.091543,2.181948,5.865708,20.089670,1.734570
6,35,3072517,b'ZTF19aclhbhx',1033505815115015004,3636,58787.505810,87.625662,28.673785,18.340000,0.036348,...,-0.008872,0.002600,-0.001374,0.036373,0.995137,0.091543,2.181943,5.804489,20.089672,1.749672
7,35,3115517,b'ZTF19acoyjha',1038496116215015008,3938,58792.496111,87.189431,28.610561,18.407499,0.043047,...,-0.008922,0.002434,-0.001386,0.043023,0.994960,0.090574,2.129219,6.495485,20.035595,1.628096
8,35,3116824,b'ZTF19acoyjha',1038499735115015006,3941,58792.499734,87.189006,28.610525,18.372499,0.043053,...,-0.008922,0.002434,-0.001386,0.043029,0.994960,0.090573,2.129182,6.499351,20.035553,1.663054
9,35,3156828,b'ZTF19acpkskg',1039499275115015008,3969,58793.499271,87.079948,28.595486,18.174500,0.044729,...,-0.008932,0.002401,-0.001388,0.044694,0.994907,0.090347,2.119157,8.422152,20.025101,1.850601
